# Q# Interop with OpenQASM

The modern QDK provides interoperability with OpenQASM 3 programs built upon the core Q# compiler infrastructure.

This core enables integration and local resource estimation without relying on external tools. Users are able to estimate resources for their OpenQASM programs locally (see the [resource estimation with Qiskit sample notebook](../../estimation/estimation-openqasm.ipynb)), leveraging the Q# compiler's capabilities for analysis, transformation, code generation, and simulation. This also enables the generation of QIR from OpenQASM progams leveraging the [modern QDKs advanced code generation capabilities](https://devblogs.microsoft.com/qsharp/integrated-hybrid-support-in-the-azure-quantum-development-kit/).

This includes support for classical instructions available in OpenQASM such as for loops, if statements, switch statements, while loops, binary expresssions, and more.

Import the Q# module.

This enables the `%%qsharp` magic and initializes a Q# interpreter singleton.

In [ ]:
import qsharp
qsharp.init(target_profile=qsharp.TargetProfile.Base)

### Run OpenQASM 3 Code in interactive session
Interactive sessions have different semantics from program execution. We no longer have inferred output and input. Instead we treat qasm lines as code fragments and interpret them one at a time (though they are all compiled together). Due to scoping rules in qasm3, all code used in the the program must be defined in the snippet and can't use compilation state from other cells or calls.

We can add an optional name parameter to compile the program into a callable operation in the interactive session.

In [ ]:
%%qasm3 --name bell
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];
bit[2] c;
c[0] = measure q[0];
c[1] = measure q[1];

With the OpenQASM program loaded into a callable name `bell`, we can now import it via the QDK's Python bindings:

In [ ]:
from qsharp.code import bell
bell()

Additionally, since it is defined in the session, we can run it directly from a Q# cell:

In [ ]:
%%qsharp
bell()

This also unlocks all of the other `qsharp` package functionality. Like noisy simulation:

In [ ]:
from qsharp_widgets import Histogram

Histogram(qsharp.run("bell()", shots=1000, noise=qsharp.DepolarizingNoise(0.01)))

Circuit rendering:

In [ ]:
qsharp.circuit(qsharp.code.bell)

Circuit widget rendering:

In [ ]:
from qsharp_widgets import Circuit
Circuit(qsharp.circuit(qsharp.code.bell))

Code generation:

In [ ]:
print(qsharp.compile(bell))

We can also define input for the compiled OpenQASM code so that we can parameterize input:

In [ ]:
%%qasm3 --name needs_args
include "stdgates.inc";
input float theta;
qubit[2] q;
rx(theta) q[0];
rx(-theta) q[1];
bit[2] c;
c[0] = measure q[0];
c[1] = measure q[1];

In [ ]:
from qsharp.code import needs_args
print(qsharp.compile(needs_args, 1.57))

Any library that exports to OpenQASM 3 can now integrate into the QDK's ecosystem.

Let's take a look at cirq for example

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install cirq
    import cirq
    print("installed cirq.")

With cirq installed, we'll build up some utilities to create a Bernstein-Vazirani circuit

In [ ]:
import random
import cirq

def make_oracle(input_qubits, output_qubit, secret_factor_bits, secret_bias_bit):
    """Gates implementing the function f(a) = a·factors + bias (mod 2)."""

    if secret_bias_bit:
        yield cirq.X(output_qubit)

    for qubit, bit in zip(input_qubits, secret_factor_bits):
        if bit:  # pragma: no cover
            yield cirq.CNOT(qubit, output_qubit)


def make_bernstein_vazirani_circuit_with_cirq(input_qubits, output_qubit, oracle):
    """Solves for factors in f(a) = a·factors + bias (mod 2) with one query."""

    c = cirq.Circuit()

    # Initialize qubits.
    c.append([cirq.X(output_qubit), cirq.H(output_qubit), cirq.H.on_each(*input_qubits)])

    # Query oracle.
    c.append(oracle)

    # Measure in X basis.
    c.append([cirq.H.on_each(*input_qubits), cirq.measure(*input_qubits, key='result')])

    return c


def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)

And we'll call the code to build the circuit

In [ ]:
import qsharp
from qsharp.interop.qasm3 import eval, estimate, import_callable
from qsharp import init
init(target_profile=qsharp.TargetProfile.Base)

qubit_count=8

# Choose qubits to use.
input_qubits = [cirq.GridQubit(i, 0) for i in range(qubit_count)]
output_qubit = cirq.GridQubit(qubit_count, 0)

# Pick coefficients for the oracle and create a circuit to query it.
secret_bias_bit = random.randint(0, 1)
secret_factor_bits = [random.randint(0, 1) for _ in range(qubit_count)]
oracle = make_oracle(input_qubits, output_qubit, secret_factor_bits, secret_bias_bit)
print(
    'Secret function:\nf(a) = '
    f"a·<{', '.join(str(e) for e in secret_factor_bits)}> + "
    f"{secret_bias_bit} (mod 2)"
)

circuit = make_bernstein_vazirani_circuit_with_cirq(input_qubits, output_qubit, oracle)

We can now see the OpenQASM 3 equivalent program

In [ ]:
qasm_str = cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0"))
print(qasm_str)

And leverage the QDK resource estimation capabilities for the cirq circuit

In [ ]:
# use QDK resource estimation on cirq circuit via QASM3
estimate(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))

If a library allows for classical controls or branching, any ouput OpenQASM they generate is consumable:

In [ ]:
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.measure(q0, key='a'),
    cirq.H(q1).with_classical_controls('a'),
    cirq.measure(q1, key='b'),
)
print(circuit)
print(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))

In [ ]:
alice = cirq.NamedQubit('alice')
bob = cirq.NamedQubit('bob')
message = cirq.NamedQubit('_message')

message_circuit = cirq.Circuit(
    # Create the message.
    cirq.X(message) ** 0.371,
    cirq.Y(message) ** 0.882,
)

circuit = cirq.Circuit(
    # Create Bell state to be shared between Alice and Bob.
    cirq.H(alice),
    cirq.CNOT(alice, bob),
    # Prepare message circuit
    message_circuit,
    # Bell measurement of the message and Alice's entangled qubit.
    cirq.CNOT(message, alice),
    cirq.H(message),
    cirq.measure(message, key='M'),
    cirq.measure(alice, key='A'),
    # Uses the two classical bits from the Bell measurement to recover the
    # original quantum message on Bob's entangled qubit.
    cirq.X(bob).with_classical_controls('A'),
    cirq.Z(bob).with_classical_controls('M'),
)
print(circuit)
print(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))

In [ ]:
import cirq
from qsharp.interop.qasm3 import estimate

q0 = cirq.LineQubit(0)
subcircuit = cirq.FrozenCircuit(cirq.measure(q0, key='a'), cirq.X(q0).with_classical_controls('a'))
circuit = cirq.Circuit(
    cirq.measure(q0, key='a'),
    cirq.CircuitOperation(subcircuit, repetitions=2),
    cirq.X(q0).with_classical_controls('a'),
)
print(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))
estimate(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))